In [1]:
import requests, bs4
import os
import re
from datetime import datetime
import sqlite3
import selenium
"""os.environ['http_proxy'] = "http://PITC-Zscaler-Americas-Cincinnati3PR.proxy.corporate.ge.com:80" 
os.environ['HTTP_PROXY'] = "http://PITC-Zscaler-Americas-Cincinnati3PR.proxy.corporate.ge.com:80"
os.environ['https_proxy'] = "http://PITC-Zscaler-Americas-Cincinnati3PR.proxy.corporate.ge.com:80"
os.environ['HTTPS_PROXY'] = "http://PITC-Zscaler-Americas-Cincinnati3PR.proxy.corporate.ge.com:80"""

'os.environ[\'http_proxy\'] = "http://PITC-Zscaler-Americas-Cincinnati3PR.proxy.corporate.ge.com:80" \nos.environ[\'HTTP_PROXY\'] = "http://PITC-Zscaler-Americas-Cincinnati3PR.proxy.corporate.ge.com:80"\nos.environ[\'https_proxy\'] = "http://PITC-Zscaler-Americas-Cincinnati3PR.proxy.corporate.ge.com:80"\nos.environ[\'HTTPS_PROXY\'] = "http://PITC-Zscaler-Americas-Cincinnati3PR.proxy.corporate.ge.com:80'

In [2]:
from selenium import webdriver
import time

driver = webdriver.Chrome()
driver.get('http://pythonscraping.com/pages/javascript/ajaxDemo.html')
time.sleep(3)
print(driver.find_element_by_id('content').text)
driver.close()

Here is some important text you want to retrieve!
A button to click!


In [72]:
from selenium import webdriver
import time

driver = webdriver.Chrome()
driver.get('https://bid.bidfta.com/cgi-bin/mnlist.cgi?broadwellv108/category/ALL')
time.sleep(3)
#driver.close()

In [73]:
#Build string of elements
build_flag = True
start_p = 'p1'
item_str = ""
while build_flag:
    try:
        item_str += driver.find_element_by_name(start_p).get_attribute("value")
        start_p = start_p[0] + str(int(start_p[1])+1)
    except:
        build_flag = False
#item_str
#driver.find_element_by_xpath("//table[@id='SelectPage']").find_element_by_name("p2").get_attribute('value')

In [74]:
elem = driver.find_elements_by_name("p2")[2]
driver.execute_script("arguments[0].setAttribute('value','" + item_str + "')", elem)

In [75]:
driver.find_element_by_xpath("//input[@value='p2'][@name='page']").click()

In [68]:
driver.close()

In [46]:
soup = bs4.BeautifulSoup(driver.page_source, 'html.parser')
date_string = soup.select('table tr td table tr td')[2].getText()
date_list = re.split("[ ,]+", date_string)
item_date = datetime.strptime(" ".join(date_list[:5]),"%B %d %Y %I:%M %p")
item_date

ValueError: time data 'Bidding extended on Open items' does not match format '%B %d %Y %I:%M %p'

In [8]:
datetime.strptime(" ".join(date_list[:5]),"%B %d %Y %I:%M %p")

datetime.datetime(2018, 4, 2, 10, 0)

In [9]:
item_details_link = r'https://bid.bidfta.com' + soup.select('a')[-3].get('href')
item_details_link

'https://bid.bidfta.com/cgi-bin/mnlist.cgi?broadwellv371/category/ALL'

In [76]:
#res = requests.get(item_details_link)
soup = bs4.BeautifulSoup(driver.page_source, 'html.parser')

In [77]:
conn = sqlite3.connect('bidfta.db')
c = conn.cursor()
c.execute('DROP TABLE items')
c.execute('''CREATE TABLE items
             (description text, additional text, uri text, current_price real)''')

In [183]:
text = str(soup.find('table',id='DataTable')
     .findAll('tr', class_='DataRow')[1]
     .findAll('td')[2]
    )
text

'<td><b>Item Brand</b>: SUNSOUT<br/><b>Item Desc</b>: OBSERVATION DECK 550 PC JIGSAW PUZZLE -NATURE THEME- BY SUNSOUT<br/><b>Load #</b>: 16178<br/><b>Lotter</b>: Mindy<br/><b>Weight</b>: 0.9<br/><b>Additional Info</b>: BRAND NEW<br/><b>Item Location</b>: Broadwell<br/><b>Front Page</b>: <a href="http://www.bidfta.com" target="_blank">Click here to go back to Fast Track Auction Home Page</a> <br/><b>Twitter</b>: <a href="https://twitter.com/home?status=Check%20out%20this%20great%20deal%20I%20found%20on%20%23FastTrackIt%20%40BidFTA%20http%3A//bid.bidfta.com/cgi-bin/mnlist.cgi?/TN071754">Twitter</a><br/><b>Facebook</b>: <a href="https://www.facebook.com/sharer/sharer.php?u=http%3A//bid.bidfta.com/cgi-bin/mnlist.cgi?/TN071754">Facebook</a><br/><b>Pinterest</b>: <a href="https://pinterest.com/pin/create/button/?url=&amp;media=http%3A//bidfta.biz//TN071754t.jpg&amp;description=Check%20out%20this%20sweet%20deal%20I%20found%20on%20%23FastTrackIt%20%40BidFTA">Pinterest</a><br/><b>Watchlist</b>:

In [153]:
mat = re.compile("<br/>")
txt = mat.split(text)[3]
mat2 = re.compile(": ([\w\W]+)$")
mat2.search(txt).groups()[0]


'BRAND NEW'

In [181]:
re.search("<b>([\w\W]+)</b>", text).group()

IndexError: no such group

In [164]:
for item in mat.split(text)[:5]:
    print(mat2.search(item).groups()[0])

SUNSOUT
OBSERVATION DECK 550 PC JIGSAW PUZZLE -NATURE THEME- BY SUNSOUT
16178
Mindy
0.9


In [ ]:
items = (soup.find('table',id='DataTable')
             .findAll('tr', class_='DataRow')
              )

for item in items:
    

In [78]:
items = soup.findAll('table')[2].findAll('table')[2].select('tr')[1:]
#print(items[0].select('td')[2])
for item in items:
    #print(item)
    price = item.select('td')[-3].getText()
    split_item_data = re.split('<b>|</b>|<br/>',str(item.select('td')[2]))
    uri = 'http://bid.bidfta.com' + item.find('a').get('href')
    for i in range(1,len(split_item_data)):
        if split_item_data[i-1] == 'Item Desc':
            item_desc = split_item_data[i][2:]
        elif split_item_data[i-1] == 'Additional Info':
            addtl = split_item_data[i][2:]
    c.execute('INSERT INTO items VALUES (?, ?, ?, ?)',(item_desc, addtl, uri, price))

In [79]:
c.execute("select * from items where description like '%assorted%'")
c.fetchall()

[('ASSORTED CELL PHONE CASES AND SCREEN PROTECTORS **LOTTER MUST OPEN BAG AND TAKE PHOTO OF 5 CASES INSIDE**',
  'BRAND NEW',
  'http://bid.bidfta.com/cgi-bin/mnlist.cgi?broadwellv108/TN071772',
  '0.11\nx 25 = 2.75'),
 ('10 PAIRS ASSORTED SUNGLASSES ',
  'BRAND NEW',
  'http://bid.bidfta.com/cgi-bin/mnlist.cgi?broadwellv108/TN071779',
  '0.69\nx 10 = 6.90'),
 ('4 ASSORTED ITEMS FOR BABY SHOWER',
  'BRAND NEW',
  'http://bid.bidfta.com/cgi-bin/mnlist.cgi?broadwellv108/TN071821',
  '0.01\nx 4 = 0.04'),
 ('3 ASSORTED CDS',
  'BRAND NEW',
  'http://bid.bidfta.com/cgi-bin/mnlist.cgi?broadwellv108/TN071823',
  0.01),
 ('2 ASSORTED PAIRS OF SOCKS ',
  'BRAND NEW',
  'http://bid.bidfta.com/cgi-bin/mnlist.cgi?broadwellv108/TN071824',
  '0.33\nx 2 = 0.66'),
 ('2 ASSORTED ITEMS ',
  'BRAND NEW',
  'http://bid.bidfta.com/cgi-bin/mnlist.cgi?broadwellv108/TN071826',
  '0.06\nx 2 = 0.12'),
 ('2 ASSORTED ITEMS ',
  'BRAND NEW',
  'http://bid.bidfta.com/cgi-bin/mnlist.cgi?broadwellv108/TN071828',
  '0

In [56]:
soup.select('table table tr')[3:-3]

[<tr class="DataRow" id="TN071803" valign="top"><td><a href="/cgi-bin/mnlist.cgi?broadwellv108/TN071803">TN071803</a></td>
 <td align="center"><a href="/cgi-bin/mnlist.cgi?broadwellv108/TN071803"><img alt="TN071803t.jpg" border="0" src="http://d2c3kiufvhjdfg.cloudfront.net/Pics/TN071803t.jpg"/></a></td><td><b>Item Desc</b>: STAINLESS STEEL FISH BASKET<br/><b>Load #</b>: 16178<br/><b>Lotter</b>: Mindy<br/><b>Additional Info</b>: BRAND NEW<br/><b>Item Location</b>: Broadwell<br/><b>Front Page</b>: <a href="http://www.bidfta.com" target="_blank">Click here to go back to Fast Track Auction Home Page</a> <br/><b>Twitter</b>: <a href="https://twitter.com/home?status=Check%20out%20this%20great%20deal%20I%20found%20on%20%23FastTrackIt%20%40BidFTA%20http%3A//bid.bidfta.com/cgi-bin/mnlist.cgi?/TN071803">Twitter</a><br/><b>Facebook</b>: <a href="https://www.facebook.com/sharer/sharer.php?u=http%3A//bid.bidfta.com/cgi-bin/mnlist.cgi?/TN071803">Facebook</a><br/><b>Pinterest</b>: <a href="https://pi

In [57]:
item_list = soup.select('table table tr')[3:-3]
item_desc_list = []

soup.select('table table tr')[-4].getText().split(': ')
split_item_data = re.split('<b>|</b>|<br/>',str(soup.select('table table tr')[-4]))

for item in soup.select('table table tr')[3:-3]:
    uri, item_desc, addtl = [None]*3
    split_item_data = re.split('<b>|</b>|<br/>',str(item))
    uri = 'http://bid.bidfta.com' + bs4.BeautifulSoup(split_item_data[0]).select('a')[0].get('href') 
    for i in range(1,len(split_item_data)):
        if split_item_data[i-1] == 'Item Desc':
            item_desc = split_item_data[i][2:]
        elif split_item_data[i-1] == 'Additional Info':
            addtl = split_item_data[i][2:]
    c.execute('INSERT INTO items VALUES (?, ?, ?)',(item_desc, addtl, uri))

C:\Users\212474523\AppData\Local\Continuum\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file C:\Users\212474523\AppData\Local\Continuum\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


OperationalError: table items has 4 columns but 3 values were supplied